#### 22K-4060 
### Ali Suleman
#### Recommender System (Assignment 2)

In [23]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
import numpy as np

In [25]:
!ls

links.csv  movies.csv  Q2.ipynb  ratings.csv  README.txt  tags.csv


In [51]:
# Load datasets
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')


In [78]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load and merge
movies_df = pd.read_csv("movies.csv")
ratings_df = pd.read_csv("ratings.csv")
merged_df = ratings_df.merge(movies_df, on='movieId')

# Make the user-item matrix
user_item_matrix = merged_df.pivot_table(index='userId', columns='title', values='rating')

In [79]:
user_item_matrix.head()

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
user_means = user_item_matrix.mean(axis=1)
user_means

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Length: 610, dtype: float64

In [81]:
adjusted_ratings = user_item_matrix.sub(user_means, axis=0)

In [82]:
# Now Applying adjusted cosine similarities on the adjusted means we computed, transpose to compute for the items
item_user_matrix = adjusted_ratings.T

item_user_matrix_filled = item_user_matrix.fillna(0)

cos_sim_matrix = cosine_similarity(item_user_matrix_filled)

similarity_matrix = pd.DataFrame(cos_sim_matrix, index=item_user_matrix.index, columns=item_user_matrix.index)

In [74]:
def predict_rating(user_id, target_item, k=5):
    user_ratings = adjusted_ratings.loc[user_id]
    if target_item not in similarity_matrix.columns:
        return np.nan

    sims = similarity_matrix[target_item]
    rated_items = user_ratings.dropna().index
    sims = sims[rated_items]

    rating_diffs = user_ratings[rated_items]

    numerator = np.sum(sims * rating_diffs)
    denominator = np.sum(np.abs(sims)) + 1e-8  # avoid division by zero
    item_mean = user_item_matrix[target_item].mean()

    return item_mean + (numerator / denominator)

# Step 6: Recommend N items for a user
def recommend_items(user_id, N=5):
    user_row = user_item_matrix.loc[user_id]
    unseen_items = user_row[user_row.isna()].index

    predictions = {
        item: predict_rating(user_id, item)
        for item in unseen_items
    }

    top_items = sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:N]
    return top_items


recommendations = recommend_items(user_id=18, N=5)
print("Top 3 movie recommendations for User 1:")
for title, score in recommendations:
    print(f"{title}: {score:.2f}")

Top 3 movie recommendations for User 1:
Entertaining Angels: The Dorothy Day Story (1996): 5.63
Animals are Beautiful People (1974): 5.62
Thousand Clowns, A (1965): 5.62
Woman Is a Woman, A (femme est une femme, Une) (1961): 5.62
Rivers and Tides (2001): 5.51
